## NER - Natasha

Natasha  - питоновская библиотека для извлечения именованных сущностей. Она похоже на Tomita-parser, но в ней все на чистом питоне, с открытым кодом и активно развивается. 

Если быть точнее, то natasha - набор готовых правил для парсера yargy. 

Есть например готовые правила для извлечения персон.

In [1]:
from natasha import NamesExtractor

In [2]:
sents = open('sents.txt').read().splitlines()

In [3]:
extractor = NamesExtractor()

Видно, что извлекается не очень хорошо. К тому же в реальной задаче нам потребуется извлекать ещё какие-то аттрибуты сущностей и группировать их в факты. И не обязательно этих типов. Поэтому полезно разобраться с самим парсером yargy.

Документация yargy: https://yargy.readthedocs.io/ru/latest/reference.html

Лучше всего с такими штуками разбираться на практике. Давайте попробуем написать правила для извлечения персон. Каждая персона должна описываться 3 полями - Имя, Фамилия, Отчество. Также у Персоны должны быть атрибуты - должность и место работы. 

За основу возьмем пример из документации:

In [4]:
from yargy import Parser, rule, and_, or_, not_
from yargy.predicates import gram
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact, attribute
from yargy.tokenizer import MorphTokenizer
from yargy.pipelines import morph_pipeline
from yargy.relations import gnc_relation
from IPython.display import display

In [5]:
Person = fact(
    'Person',
    ['position', 'name']
)
Name = fact(
    'Name',
    ['first', 'last']
)


POSITION = morph_pipeline([
    'премьер министр',
    'президент'
])

NAME = rule(
    gram('Name').interpretation(
        Name.first.inflected()
    ),
    gram('Surn').interpretation(
        Name.last.inflected()
    )
).interpretation(
    Name
)

PERSON = rule(
    POSITION.interpretation(
        Person.position.inflected()
    ),
    NAME.interpretation(
        Person.name
    )
).interpretation(
    Person
)


parser = Parser(PERSON)

Посмотрим, что получается:

In [6]:
for sent in sents[:100]:
    for match in parser.findall(sent):
        print(sent)
        display(match.fact)
        print('---------------')

По слухам, министр внутренних дел Исгендер Муликов сообщил президенту Гурбангулы Бердымухамедову о частых ДТП с участием женщин.


Person(position='президент',
       name=Name(first='гурбангул',
                 last='бердымухамедов'))

---------------


Чтобы проверить какие морфологические тэги приписываются словам, можно использовать MorphTokenizer.

In [7]:
tokenizer = MorphTokenizer()

In [8]:
list(tokenizer('Медведев'))

[MorphToken('Медведев',
            [0, 8),
            'RU',
            [Form('медведев', Grams(NOUN,Sgtm,Surn,anim,masc,nomn,sing))])]

In [9]:
list(tokenizer('евроинтеграции'))

[MorphToken('евроинтеграции',
            [0, 14),
            'RU',
            [Form('евроинтеграция', Grams(NOUN,femn,gent,inan,sing)),
             Form('евроинтеграция', Grams(NOUN,datv,femn,inan,sing)),
             Form('евроинтеграция', Grams(NOUN,femn,inan,loct,sing)),
             Form('евроинтеграция', Grams(NOUN,femn,inan,nomn,plur)),
             Form('евроинтеграция', Grams(NOUN,accs,femn,inan,plur))])]

## Задача на семинар:
Доработать правила так, чтобы извлекалось как можно больше правильных фактов.

In [10]:
with open('prof.txt', 'r', encoding='utf-8') as f:
    prof = f.readlines()

In [11]:
prof = [elem.strip() for elem in prof]

In [12]:
gnc = gnc_relation()

Name = fact(
    'Name',
    ['first', 'last'],
)
Person = fact(
    'Person',
    ['position', 'name']
)

LAST = and_(
    gram('Surn'),
    not_(gram('Abbr')),
)
FIRST = and_(
    gram('Name'),
    not_(gram('Abbr')),
)

POSITION = morph_pipeline(prof)

gnc = gnc_relation()
NAME = rule(
    FIRST.interpretation(
        Name.first.inflected()
    ).optional().match(gnc),
    LAST.interpretation(
        Name.last.inflected()
    ).match(gnc)
).interpretation(Name)

PERSON = or_(
    rule(
    POSITION.interpretation(
        Person.position.inflected()
    ),
    gram('Geox').optional().repeatable(),
    NAME.interpretation(
        Person.name
    )),
    rule(
    POSITION.interpretation(
        Person.position.inflected()
    ),
    gram('gent').optional().repeatable(),
    NAME.interpretation(
        Person.name.inflected()
    )),
).interpretation(
    Person
)
   
parser = Parser(PERSON)

In [13]:
i = 1

for sent in sents:
    for match in parser.findall(sent):
        print(i, '---------------')
        print(sent)
        print([_.value for _ in match.tokens])
        try:  
            display(match.fact)
        except:
            pass
        i += 1

1 ---------------
Напомним, что премьер-министр России Дмитрий Медведев подписал изменения в Правилах дорожного движения.
['премьер', '-', 'министр', 'России', 'Дмитрий', 'Медведев']


Person(position='премьер-министр',
       name=Name(first='дмитрий',
                 last='медведев'))

2 ---------------
Министр финансов Греции Янис Варуфакис еще днем заявил: «Греция не будет переводить платеж МВФ сегодня».
['Министр', 'финансов', 'Греции', 'Янис', 'Варуфакис']
3 ---------------
Премьер Зубков, решая судьбу Чаянды, был поставлен перед выбором: поступить ли по закону или исходя из политической целесообразности.
['Премьер', 'Зубков']


Person(position='премьер',
       name=Name(first=None,
                 last='зубков'))

4 ---------------
В список среди прочих попали премьер-министр Дмитрий Медведев и все члены кабинета министров, а также 43 сотрудника администрации президента РФ, включая пресс-секретаря Дмитрия Пескова, главу администрации Антона Вайно, советников и помощников главы государства.
['премьер', '-', 'министр', 'Дмитрий', 'Медведев']


...

62 ---------------
Президент РФ Владимир Путин провел встречи с главами государств АСЕАН, продемонстрировав готовность к энергичному диалогу на самом высоком уровне.
['Президент', 'РФ', 'Владимир', 'Путин']


Person(position='президент',
       name=Name(first='владимир',
                 last='путин'))

63 ---------------
Это же отмечал министр экономического развития РФ Максим Орешкин, выступая в конце ноября прошлого года на международной конференции «Инвестиции в климат: новая парадигма экономического развития России», где он подчеркнул, что и российская, и мировая экономика должна адаптироваться к климатическим изменениям.
['министр', 'экономического', 'развития', 'РФ', 'Максим', 'Орешкин']
64 ---------------
Выступая в баварской столице в субботу, глава МИД РФ Сергей Лавров вновь опроверг эти обвинения.
['глава', 'МИД', 'РФ', 'Сергей', 'Лавров']
65 ---------------
По словам постпреда РФ при ООН Виталия Чуркина, речь идет об установлении с 1 марта режима прекращения огня между противоборствующими группировками.
['постпреда', 'РФ', 'при', 'ООН', 'Виталия', 'Чуркина']
66 ---------------
По данным начальника ГОУ ГШ ВС РФ генерал-полковника Сергея Рудского, наутро 17 марта из Восточной Гуты было выведено 44 639 человек.
['генерал', '-', 'полковника', 'Сергея', 'Рудского']


Person(position='генерал-полковник',
       name=Name(first='сергей',
                 last='рудский'))

67 ---------------
«Сегодня в ходе первого массированного авиационного удара 34 крылатыми ракетами воздушного базирования большой дальности уничтожено 14 важных объектов террористов – это пункты управления незаконными вооруженными формированиями, осуществлявшие координацию действий отрядов боевиков ИГИЛ в провинциях Идлиб и Алеппо», – сказал на совещании во вторник начальник Генштаба ВС РФ Валерий Герасимов.
['начальник', 'Генштаба', 'ВС', 'РФ', 'Валерий', 'Герасимов']
68 ---------------
Об этом сообщил на расширенной коллегии в Минобрнауки РФ глава Рособрнадзора Сергей Кравцов.
['глава', 'Рособрнадзора', 'Сергей', 'Кравцов']


In [15]:
i - 1

68